In [ ]:
import os
import json
import sagemaker
import numpy as np
import pandas as pd
import tensorflow as tf
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

# Gets number of users and pages from stored data (data prep)
%store -r n_user
%store -r n_item

# Get bucket/sagemaker session data
sage_session = sagemaker.Session()
role = get_execution_role()
bucket = sage_session.default_bucket()

# Location for training dataset
training_uri = os.path.join(f's3://{bucket_name}', 'data')

# Sagemaker instance data subject to change after more research
num_of_instance = 1
instance_type = 'ml.c5.2xlarge'
train_script = "script.py"

# Some base parameters too be tweaked later
training_parameters = {
    'epochs': 1,
    'batch_size': 256,
    'n_user': n_user,
    'n_item': n_item
}

# training framework specs
tensorflow_version = '2.1.0'
python_version = 'py3'
distributed_training_spec = {'parameter_server': {'enabled': True}}

# Instantiate training job
ncf_estimator = TensorFlow(
    entry_point=train_script,
    role=role,
    train_instance_count=num_of_instance,
    train_instance_type=instance_type,
    framework_version=tensorflow_version,
    py_version=python_version,
    distributions=distributed_training_spec,
    hyperparameters=training_parameters
)

# Start training
ncf_estimator.fit(training_data_uri)

# Deploying of training model so it can be called
endpoint_name = 'odb-recommendation-engine'

predictor = ncf_estimator.deploy(initial_instance_count=1,
                                 instance_type='ml.c5.xlarge',
                                 endpoint_type='tensorflow-serving',
                                 endpoint_name=endpoint_name)
